In [1]:
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import rasterio
from tqdm import tqdm
import pickle

# Helper functions
def read(file):
    with rasterio.open(file) as src:
        return src.read(), src.profile

def get_dates(path, n=None):
    files = os.listdir(path)
    dates = list()
    for f in files:
        f = f.split("_")[0]
        if len(f) == 8:  # Check for valid date format e.g., 20160110
            dates.append(f)
    dates = set(dates)
    if n is not None:
        dates = random.sample(dates, n)
    dates = list(dates)
    dates.sort()
    return dates

# Define the dataset class
class SentinelDataset(Dataset):
    def __init__(self, root_dirs, seqlength=30, preprocess_dirs=None, save_preprocessed=False):
        self.root_dirs = [r.rstrip("/") for r in root_dirs]
        self.seqlength = seqlength
        self.samples = list()
        self.ndates = list()
        self.classids, self.classes = self.read_classes()
        self.preprocess_dirs = preprocess_dirs
        self.save_preprocessed = save_preprocessed
        self.prepare_dataset()

    def read_classes(self):
        classes_path = 'data/sentinel2/lombardia-classes/classes25pc.txt'
        with open(classes_path, 'r') as f:
            classes = f.readlines()
        ids, names = [], []
        for row in classes:
            row = row.replace("\n", "")
            if '|' in row:
                cls_info = row.split('|')
                id_info = [int(x) for x in cls_info[0].split(',')]
                ids.append(id_info)
                names.append(cls_info[1])
        return ids, names

    def prepare_dataset(self):
        for root_dir, preprocess_dir in zip(self.root_dirs, self.preprocess_dirs):
            print("Reading dataset info:", root_dir)
            dirs = []
            for year_dir in os.listdir(root_dir):
                year_path = os.path.join(root_dir, year_dir)
                for tile_dir in os.listdir(year_path):
                    tile_path = os.path.join(year_path, tile_dir)
                    if os.path.isdir(tile_path):
                        dirs.append(tile_path)

            stats = dict(rejected_nopath=0, rejected_length=0, total_samples=0)
            for path in tqdm(dirs):
                if not os.path.exists(path) or not os.path.exists(os.path.join(path, "20160110.tif")):
                    stats["rejected_nopath"] += 1
                    continue
                ndates = len(get_dates(path))
                if ndates < self.seqlength:
                    stats["rejected_length"] += 1
                    continue
                stats["total_samples"] += 1
                self.samples.append((path, preprocess_dir))
                self.ndates.append(ndates)
            self.print_stats(stats)
            if self.save_preprocessed:
                self.save_preprocessed_data()

    def save_preprocessed_data(self):
        for sample, preprocess_dir in self.samples:
            label, profile = read(os.path.join(sample, "y.tif"))
            dates = get_dates(sample, n=self.seqlength)
            data = {
                'label': label,
                'profile': profile,
                'dates': dates
            }
            save_path = os.path.join(preprocess_dir, os.path.basename(sample) + ".pkl")
            os.makedirs(preprocess_dir, exist_ok=True)
            try:
                with open(save_path, 'wb') as f:
                    pickle.dump(data, f)
                print(f"Saved preprocessed data to {save_path}")
            except PermissionError as e:
                print(f"Permission error: {e}")
            except FileNotFoundError as e:
                print(f"File not found: {e}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample_path, preprocess_dir = self.samples[idx]
        label, profile = read(os.path.join(sample_path, "y.tif"))
        dates = get_dates(sample_path, n=self.seqlength)
        data_10m, data_20m, data_60m = [], [], []
        for date in dates:
            data_10m.append(read(os.path.join(sample_path, date + "_10m.tif"))[0])
            data_20m.append(read(os.path.join(sample_path, date + "_20m.tif"))[0])
            data_60m.append(read(os.path.join(sample_path, date + "_60m.tif"))[0])
        data_10m = np.array(data_10m) * 1e-4
        data_20m = np.array(data_20m) * 1e-4
        data_60m = np.array(data_60m) * 1e-4
        data_20m = torch.from_numpy(data_20m)
        data_60m = torch.from_numpy(data_60m)
        data_20m = torch.nn.functional.interpolate(data_20m, size=data_10m.shape[2:4])
        data_60m = torch.nn.functional.interpolate(data_60m, size=data_10m.shape[2:4])
        data = torch.cat((torch.from_numpy(data_10m), data_20m, data_60m), 1)
        data = data.permute(1, 0, 2, 3).float()
        label = torch.from_numpy(label).long()
        return data, label

    def print_stats(self, stats):
        print_lst = list()
        for k, v in zip(stats.keys(), stats.values()):
            print_lst.append("{}:{}".format(k, v))
        print('\n', ", ".join(print_lst))

# Define paths and parameters
root_paths = ['data/sentinel2/lombardia', 'data/sentinel2/lombardia2']
preprocess_dirs = ['data/sentinel2/lombardia_preprocessed', 'data/sentinel2/lombardia2_preprocessed']

# Initialize and prepare dataset
dataset = SentinelDataset(root_paths, seqlength=30, preprocess_dirs=preprocess_dirs, save_preprocessed=True)

# Create DataLoader for training and validation
batch_size = 16
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Verify DataLoader
for batch in train_loader:
    inputs, labels = batch
    print('Input shape:', inputs.shape)
    print('Label shape:', labels.shape)
    break

Reading dataset info: data/sentinel2/lombardia


100%|██████████| 13312/13312 [00:06<00:00, 2006.78it/s]
C:\Users\Richa\AppData\Local\Temp\ipykernel_18552\4253084667.py:24: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  dates = random.sample(dates, n)



 rejected_nopath:9984, rejected_length:0, total_samples:3328
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\10.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\100.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1000.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1001.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1002.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1003.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1004.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1005.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1006.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1007.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1008.pkl
Saved preprocessed data to data/sentinel2/lombardia_prep

100%|██████████| 13312/13312 [00:05<00:00, 2547.00it/s]



 rejected_nopath:13312, rejected_length:0, total_samples:0
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\10.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\100.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1000.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1001.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1002.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1003.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1004.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1005.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1006.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1007.pkl
Saved preprocessed data to data/sentinel2/lombardia_preprocessed\1008.pkl
Saved preprocessed data to data/sentinel2/lombardia_prepro

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50

class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ASPP, self).__init__()
        self.atrous_block1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.atrous_block6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=6, dilation=6, bias=False)
        self.atrous_block12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=12, dilation=12, bias=False)
        self.atrous_block18 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=18, dilation=18, bias=False)
        self.conv_1x1_output = nn.Conv2d(out_channels * 4, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        x1 = self.atrous_block1(x)
        x2 = self.atrous_block6(x)
        x3 = self.atrous_block12(x)
        x4 = self.atrous_block18(x)
        x = torch.cat((x1, x2, x3, x4), dim=1)
        x = self.conv_1x1_output(x)
        x = self.batch_norm(x)
        return self.relu(x)

class DeepLabV3(nn.Module):
    def __init__(self, num_classes, in_channels=3, pretrained=True):
        super(DeepLabV3, self).__init__()
        self.backbone = resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])
        self.aspp = ASPP(2048, 256)
        self.conv1 = nn.Conv2d(256, 256, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(256)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(256, num_classes, kernel_size=1)

    def forward(self, x):
        x = self.backbone(x)
        x = self.aspp(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        return F.interpolate(x, scale_factor=8, mode='bilinear', align_corners=True)

# Define number of classes based on your dataset
num_classes = 22
model = DeepLabV3(num_classes=num_classes, in_channels=3, pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


In [3]:
import optuna
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameters to tune
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
    gamma = trial.suggest_float('gamma', 0.1, 0.9)
    step_size = trial.suggest_int('step_size', 1, 10)
    batch_size = trial.suggest_int('batch_size', 8, 32)
    
    # Create DataLoader with the trial's batch size
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    # Initialize the model, criterion, optimizer, and scheduler with trial's hyperparameters
    model = DeepLabV3(num_classes=num_classes, in_channels=3, pretrained=True)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    criterion = nn.CrossEntropyLoss()
    
    # Training loop for a few epochs
    num_epochs = 5  # You can adjust this for faster tuning
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        scheduler.step()
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
    
    val_loss /= len(val_loader.dataset)
    return val_loss

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print("Best parameters:", study.best_params)
print("Best validation loss:", study.best_value)

# Best hyperparameters from Optuna
best_params = study.best_params

# Create DataLoader with the best batch size
train_loader = DataLoader(dataset, batch_size=best_params['batch_size'], shuffle=True, num_workers=4)
val_loader = DataLoader(dataset, batch_size=best_params['batch_size'], shuffle=False, num_workers=4)

# Initialize the model, criterion, optimizer, and scheduler with best hyperparameters
model = DeepLabV3(num_classes=num_classes, in_channels=3, pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])
scheduler = StepLR(optimizer, step_size=best_params['step_size'], gamma=best_params['gamma'])

# Move model to the appropriate device
model.to(device)

# Print model architecture to verify
print(model)


[I 2024-07-03 20:44:40,134] A new study created in memory with name: no-name-74537ca6-06e5-4613-a92e-e4a43421ea3a
c:\Users\Richa\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Richa\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / total

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    
    # Adjust the learning rate
    scheduler.step()

print('Training complete')


NameError: name 'study' is not defined

In [ ]:
# Evaluate the model on the validation set
model.eval()
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

val_loss /= len(val_loader.dataset)
val_accuracy = correct / total

print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Save the model
torch.save(model.state_dict(), 'best_model.pth')
print('Model saved to best_model.pth')

